In [91]:
from pyspark.sql import SparkSession

In [92]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [93]:
from pyspark.ml.regression import LinearRegression

# Data Loading

In [94]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,header=True)


In [95]:
# Data Understanding

In [96]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [97]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [98]:
for item in data.head(2)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [99]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [100]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [101]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                               outputCol='features')

In [102]:
output = assembler.transform(data)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [103]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [104]:
final_data = output.select('features','Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [105]:
train_data,test_data = final_data.randomSplit([0.7,0.3],seed=4)

In [106]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean|  497.3346792895718|
| stddev|  80.71700467837536|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [107]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean|  504.0671927896366|
| stddev|  75.89916196339628|
|    min| 256.67058229005585|
|    max|  708.9351848669818|
+-------+-------------------+



# Model

In [108]:
lr = LinearRegression(labelCol='Yearly Amount Spent',featuresCol='features')

In [109]:
lr_model = lr.fit(train_data)

In [110]:
test_results = lr_model.evaluate(test_data)

In [111]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|   8.961103480937481|
|  -4.407082613984755|
| -22.278895312276177|
|  10.088747562772369|
|  -4.426291026806439|
|  3.4220291954751474|
|-0.37301125396743373|
|  2.2859391737474084|
|  -6.182459945625169|
|  3.4077225468025176|
| -14.988278123598548|
|  -2.713046591447551|
|  17.513462300305264|
|  16.782706006682986|
|  -8.071552616655367|
| -12.169098712792675|
| -17.764067072092132|
|   -9.75608684352045|
|  -10.47469473478725|
|  -17.77222192116716|
+--------------------+
only showing top 20 rows



In [112]:
test_results.rootMeanSquaredError

9.807625717477778

In [113]:
test_results.r2

0.983188042434676

In [114]:
test_results.r2adj

0.9827144661652302

In [115]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



# deploy the model

In [116]:
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.8794843441274...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.3584771924370...|
|[31.3895854806643...|
|[31.4459724827577...|
|[31.5147378578019...|
|[31.5316044825729...|
|[31.5741380228732...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.7207699002873...|
|[31.8093003166791...|
|[31.8164283341993...|
|[31.8279790554652...|
|[31.8648325480987...|
|[31.9048571310136...|
+--------------------+
only showing top 20 rows



In [117]:
prediction = lr_model.transform(unlabeled_data)
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|   399.67924759169|
|[30.8794843441274...|494.61368259883943|
|[31.1239743499119...|509.22594915204195|
|[31.1695067987115...|417.26778323952044|
|[31.2681042107507...|427.89682420063036|
|[31.3584771924370...|491.75392125400026|
|[31.3895854806643...| 410.4426223139503|
|[31.4459724827577...|482.59102576138116|
|[31.5147378578019...| 495.9949479420866|
|[31.5316044825729...|433.10788318256004|
|[31.5741380228732...| 559.3975502841854|
|[31.5761319713222...| 543.9396305807759|
|[31.6005122003032...|461.65938919079167|
|[31.6098395733896...| 427.7628436444252|
|[31.7207699002873...| 546.8464860946783|
|[31.8093003166791...| 548.9409980756338|
|[31.8164283341993...| 518.8865585757485|
|[31.8279790554652...|449.75883439046197|
|[31.8648325480987...| 450.3659752116009|
|[31.9048571310136...| 491.7220793439833|
+--------------------+------------